In [12]:

import json

"""Models"""

from Model import Model
#from ModelNewES import ModelNewES
#from ModelNewWD import ModelNewWD


#modelNewES = ModelNewES()

#modelNewWD = ModelNewWD()

modelIBM = Model("IBM.h5")
#We must call this cause of a keras bug
#https://github.com/keras-team/keras/issues/2397
#modelIBM.label("Therefore fixed punishment will")



IBM.h5


NotImplementedError: Cannot convert a symbolic Tensor (shared_varLSTM_1/forward_lstm_1/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [ ]:
class ClassifyIBM(Resource):
    def post(self):
       """
       Classifies input text to argument structure (IBM model, fasttext - big dataset)
       ---
       consumes:
         - text/plain
       parameters:
         - in: body
           name: text
           type: string
           required: true
           description: Text to classify
           example: Quebecan independence is justified. In the special episode in Japan, his system is restored by a doctor who wishes to use his independence for her selfish reasons.
       responses:
         200:
           description: A list of tagged tokens annotated with labels
           schema:
             id: argument-structure
             properties:
               argument-structure:
                 type: string
                 description: JSON-List
                 default: No input text set
        """
       inputtext = request.get_data().decode('UTF-8')
       result = modelIBM.label_with_probs(inputtext)
       response = make_response(jsonify(result))
       response.headers['content-type'] = 'application/json'
       return response


In [14]:
def do_label_arg(marks):
    # print("marks:" + str(marks))
    marks_new = []
    for i, item in enumerate(marks):
        # for (var i = 0; i < marks.length; i++):
        if i > 0 and i + 1 < len(marks):
            # Start Label
            if marks[i]['label'][0] == "P" and marks[i - 1]['label'][0] != marks[i]['label'][0]:
                mark = {'type': "PREMISE", 'start': marks[i]['start']}
                marks_new.append(mark)
            elif marks[i]['type'][0] == "C" and marks[i - 1]['type'][0] != marks[i]['type'][0]:
                mark = {'type': "CLAIM", 'start': marks[i]['start']}
                marks_new.append(mark)
                # End Label
            if marks[i]['type'][0] == "P" or marks[i]['type'][0] == "C":
                if marks[i]['type'][0] != marks[i + 1]['type'][0]:
                    mark = marks_new.pop()
                    mark['end'] = marks[i]['end']
                    marks_new.append(mark)
        elif i == 0 and i + 1 < len(marks):
            # Start Label
            if marks[i]['type'][0] == "P":
                mark = {'type': "PREMISE", 'start': marks[i]['start']}
                marks_new.append(mark)
            elif (marks[i]['type'][0] == "C"):
                mark = {'type': "CLAIM", 'start': marks[i]['start']}
                marks_new.append(mark)
            # End Label
            if marks[i]['type'][0] == "P" or marks[i]['type'][0] == "C":
                if marks[i]['type'][0] != marks[i + 1]['type'][0]:
                    mark = marks_new.pop()
                    mark['end'] = marks[i]['end']
                    marks_new.append(mark)
        elif i == 0 and i + 1 == len(marks):
            # End Label
            if marks[i]['type'][0] == "P" or marks[i]['type'][0] == "C":
                mark['end'] = marks[i]['end']
                marks_new.append(mark)
    return marks_new

In [8]:
result = [[{"label": "C-B", "prob": "1.0", "token": "Quebecan"}, {"label": "C-I", "prob": "1.0", "token": "independence"}, {"label": "C-I", "prob": "1.0", "token": "is"}, {"label": "C-I", "prob": "1.0", "token": "justified"}, {"label": "O", "prob": "1.0", "token": "."}], [{"label": "P-B", "prob": "0.99999774", "token": "In"}, {"label": "P-I", "prob": "0.99999356", "token": "the"}, {"label": "P-I", "prob": "0.9999987", "token": "special"}, {"label": "P-I", "prob": "0.9999993", "token": "episode"}, {"label": "P-I", "prob": "0.9999993", "token": "in"}, {"label": "P-I", "prob": "0.9999995", "token": "Japan"}, {"label": "O", "prob": "1.0", "token": ","}, {"label": "P-I", "prob": "0.9999994", "token": "his"}, {"label": "P-I", "prob": "0.9999993", "token": "system"}, {"label": "P-I", "prob": "0.99999917", "token": "is"}, {"label": "P-I", "prob": "0.9999995", "token": "restored"}, {"label": "P-I", "prob": "0.9999994", "token": "by"}, {"label": "P-I", "prob": "0.9999993", "token": "a"}, {"label": "P-I", "prob": "0.9999994", "token": "doctor"}, {"label": "P-I", "prob": "0.9999994", "token": "who"}, {"label": "P-I", "prob": "0.9999994", "token": "wishes"}, {"label": "P-I", "prob": "0.99999964", "token": "to"}, {"label": "P-I", "prob": "0.99999917", "token": "use"}, {"label": "P-I", "prob": "0.99999976", "token": "his"}, {"label": "P-I", "prob": "0.9999993", "token": "independence"}, {"label": "P-I", "prob": "0.99999964", "token": "for"}, {"label": "P-I", "prob": "0.99999964", "token": "her"}, {"label": "P-I", "prob": "0.9999994", "token": "selfish"}, {"label": "P-I", "prob": "0.9999994", "token": "reasons"}, {"label": "O", "prob": "0.99999976", "token": "."}]]
data = []
text = "Quebecan independence is justified. In the special episode in Japan, his system is restored by a doctor who wishes to use his independence for her selfish reasons."
currentPos = 0
for sentence in result:
    for token in sentence:
        start = text.find(token["token"], currentPos)
        end = start + len(token["token"])
        currentPos = end
        currentWord = {}
        currentWord['start'] = start
        currentWord['end'] = end
        currentWord['type'] = token["label"]
        data.append(currentWord)
print(data)

[{'start': 0, 'end': 8, 'type': 'C-B'}, {'start': 9, 'end': 21, 'type': 'C-I'}, {'start': 22, 'end': 24, 'type': 'C-I'}, {'start': 25, 'end': 34, 'type': 'C-I'}, {'start': 34, 'end': 35, 'type': 'O'}, {'start': 36, 'end': 38, 'type': 'P-B'}, {'start': 39, 'end': 42, 'type': 'P-I'}, {'start': 43, 'end': 50, 'type': 'P-I'}, {'start': 51, 'end': 58, 'type': 'P-I'}, {'start': 59, 'end': 61, 'type': 'P-I'}, {'start': 62, 'end': 67, 'type': 'P-I'}, {'start': 67, 'end': 68, 'type': 'O'}, {'start': 69, 'end': 72, 'type': 'P-I'}, {'start': 73, 'end': 79, 'type': 'P-I'}, {'start': 80, 'end': 82, 'type': 'P-I'}, {'start': 83, 'end': 91, 'type': 'P-I'}, {'start': 92, 'end': 94, 'type': 'P-I'}, {'start': 95, 'end': 96, 'type': 'P-I'}, {'start': 97, 'end': 103, 'type': 'P-I'}, {'start': 104, 'end': 107, 'type': 'P-I'}, {'start': 108, 'end': 114, 'type': 'P-I'}, {'start': 115, 'end': 117, 'type': 'P-I'}, {'start': 118, 'end': 121, 'type': 'P-I'}, {'start': 122, 'end': 125, 'type': 'P-I'}, {'start': 1

In [15]:
print(do_label_arg(result))

TypeError: list indices must be integers or slices, not str

In [42]:
def get_argument_relation(labeling_output):
    """ post processing
    it takes the output of targer seqeunce labling and returns:
    a: segements based on BIO labling scheme
    b: argument relation between a pair of proposition - it is based on a rather simple assumption 
    that if one of the proposition is a premise wheile the other is the claim,
    an argument relation is introduced between the two"""
    
    props = {}
    for prop in labeling_output:
        segment = []
        prop_type = ""
        for token in prop:
            if token['label'] in ['P-B', 'P-I']:
                segment.append(token['token'])
                prop_type = "premise"
            elif token['label'] in ['C-B', 'C-I']:
                segment.append(token['token'])
                prop_type = "Claim"
        props[' '.join(segment)] = prop_type
    print(props.keys())
    p1 = list(props.keys())[0]
    p1_type = props[p1]
    p2 = list(props.keys())[1]
    p2_type = props[p2]


    return props, p1!=p2

def get_segments(labeling_output):
    """ post processing
    it takes the output of targer seqeunce labling and returns:
    a: segements based on BIO labling scheme
    b: argument relation between a pair of proposition - it is based on a rather simple assumption 
    that if one of the proposition is a premise wheile the other is the claim,
    an argument relation is introduced between the two"""
    
    props = {}
    for prop in labeling_output:
        segment = []
        prop_type = ""
        for token in prop:
            if token['label'] in ['P-B', 'P-I']:
                segment.append(token['token'])
                prop_type = "premise"
            elif token['label'] in ['C-B', 'C-I']:
                segment.append(token['token'])
                prop_type = "Claim"
        props[' '.join(segment)] = prop_type
    return props

In [43]:
print(get_argument_relation(result))

dict_keys(['Quebecan independence is justified', 'In the special episode in Japan his system is restored by a doctor who wishes to use his independence for her selfish reasons'])
({'Quebecan independence is justified': 'Claim', 'In the special episode in Japan his system is restored by a doctor who wishes to use his independence for her selfish reasons': 'premise'}, True)


In [44]:
result2 = [[{"label": "P-B", "prob": "0.9997757", "token": "Tim"}, {"label": "P-I", "prob": "0.9999759", "token": "Stanley"}, {"label": "P-I", "prob": "0.999974", "token": ":"}, {"label": "P-I", "prob": "0.9999976", "token": "Giles"}, {"label": "P-I", "prob": "0.9999994", "token": "name"}, {"label": "P-I", "prob": "0.9999995", "token": "for"}, {"label": "P-I", "prob": "0.9999995", "token": "me"}, {"label": "P-I", "prob": "0.9999995", "token": "please"}, {"label": "P-I", "prob": "0.99999917", "token": "one"}, {"label": "P-I", "prob": "0.9999994", "token": "thing"}, {"label": "P-I", "prob": "0.9999989", "token": "that"}, {"label": "P-I", "prob": "0.9999995", "token": "in"}, {"label": "P-I", "prob": "0.99999964", "token": "your"}, {"label": "P-I", "prob": "0.9999994", "token": "private"}, {"label": "P-I", "prob": "0.9999995", "token": "life"}, {"label": "P-I", "prob": "0.9999989", "token": "is"}, {"label": "P-I", "prob": "0.9999993", "token": "utterly"}, {"label": "P-I", "prob": "0.9999994", "token": "immoral"}, {"label": "P-I", "prob": "0.99999845", "token": "but"}, {"label": "P-I", "prob": "0.9999994", "token": "in"}, {"label": "P-I", "prob": "0.9999995", "token": "politics"}, {"label": "P-I", "prob": "0.9999989", "token": "would"}, {"label": "P-I", "prob": "0.99999917", "token": "be"}, {"label": "P-I", "prob": "0.99999964", "token": "regarded"}, {"label": "P-I", "prob": "0.9999995", "token": "as"}, {"label": "P-I", "prob": "0.9999995", "token": "a"}, {"label": "P-I", "prob": "0.99999917", "token": "virtue"}, {"label": "P-I", "prob": "0.99999857", "token": "?"}], [{"label": "P-B", "prob": "0.9997049", "token": "Giles"}, {"label": "P-I", "prob": "0.999979", "token": "Fraser"}, {"label": "O", "prob": "0.6769028", "token": ":"}, {"label": "P-I", "prob": "0.99999666", "token": "Well"}, {"label": "P-I", "prob": "0.9999993", "token": "he"}, {"label": "P-I", "prob": "0.9999994", "token": "said"}, {"label": "P-I", "prob": "0.99999917", "token": "that"}, {"label": "P-I", "prob": "0.9999993", "token": "ruthlessness"}, {"label": "P-I", "prob": "0.9999993", "token": "was"}, {"label": "P-I", "prob": "0.99999917", "token": "a"}, {"label": "P-I", "prob": "0.9999994", "token": "classic"}, {"label": "P-I", "prob": "0.9999995", "token": "example"}, {"label": "P-I", "prob": "0.99999917", "token": "and"}, {"label": "P-I", "prob": "0.9999993", "token": "thats"}, {"label": "P-I", "prob": "0.9999995", "token": "what"}, {"label": "P-I", "prob": "0.99999905", "token": "he"}, {"label": "P-I", "prob": "0.9999994", "token": "focused"}, {"label": "P-I", "prob": "0.9999994", "token": "on"}, {"label": "O", "prob": "0.99999976", "token": "."}], [{"label": "P-B", "prob": "0.9993655", "token": "So"}, {"label": "P-I", "prob": "0.99989545", "token": "ruthlessness"}, {"label": "P-I", "prob": "0.99999154", "token": "is"}, {"label": "P-I", "prob": "0.999997", "token": "not"}, {"label": "P-I", "prob": "0.9999993", "token": "something"}, {"label": "P-I", "prob": "0.9999987", "token": "that"}, {"label": "P-I", "prob": "0.9999982", "token": "is"}, {"label": "P-I", "prob": "0.9999994", "token": "a"}, {"label": "P-I", "prob": "0.9999989", "token": "domestic"}, {"label": "P-I", "prob": "0.9999994", "token": "virtue"}, {"label": "P-I", "prob": "0.9999995", "token": "but"}, {"label": "P-I", "prob": "0.99999964", "token": "it"}, {"label": "P-I", "prob": "0.9999993", "token": "may"}, {"label": "P-I", "prob": "0.99999964", "token": "well"}, {"label": "P-I", "prob": "0.9999995", "token": "be"}, {"label": "P-I", "prob": "0.99999964", "token": "a"}, {"label": "P-I", "prob": "0.9999995", "token": "public"}, {"label": "P-I", "prob": "0.9999995", "token": "one"}, {"label": "P-I", "prob": "0.9999987", "token": "because"}, {"label": "P-I", "prob": "0.9999995", "token": "there"}, {"label": "P-I", "prob": "0.9999994", "token": "are"}, {"label": "P-I", "prob": "0.9999995", "token": "times"}, {"label": "P-I", "prob": "0.9999995", "token": "in"}, {"label": "P-I", "prob": "0.9999995", "token": "which"}, {"label": "P-I", "prob": "0.9999994", "token": "you"}, {"label": "P-I", "prob": "0.9999987", "token": "have"}, {"label": "P-I", "prob": "0.99999964", "token": "to"}, {"label": "P-I", "prob": "0.9999994", "token": "do"}, {"label": "P-I", "prob": "0.99999964", "token": "things"}, {"label": "P-I", "prob": "0.99999905", "token": "that"}, {"label": "P-I", "prob": "0.99999917", "token": "youre"}, {"label": "P-I", "prob": "0.9999993", "token": "just"}, {"label": "O", "prob": "0.99999976", "token": "."}], [{"label": "P-B", "prob": "0.9998317", "token": "Tim"}, {"label": "P-I", "prob": "0.99998164", "token": "Stanley"}, {"label": "P-I", "prob": "0.9999689", "token": ":"}, {"label": "P-I", "prob": "0.9999919", "token": "But"}, {"label": "P-I", "prob": "0.9999995", "token": "the"}, {"label": "P-I", "prob": "0.9999995", "token": "public"}, {"label": "P-I", "prob": "0.9999995", "token": "and"}, {"label": "P-I", "prob": "0.9999995", "token": "the"}, {"label": "P-I", "prob": "0.9999995", "token": "private"}, {"label": "P-I", "prob": "0.9999994", "token": "are"}, {"label": "P-I", "prob": "0.9999995", "token": "not"}, {"label": "P-I", "prob": "0.9999994", "token": "separate"}, {"label": "O", "prob": "0.99999976", "token": "."}], [{"label": "O", "prob": "1.0", "token": "The"}, {"label": "P-B", "prob": "0.9999757", "token": "more"}, {"label": "P-I", "prob": "0.99999225", "token": "the"}, {"label": "P-I", "prob": "0.99999595", "token": "public"}, {"label": "P-I", "prob": "0.9999995", "token": "people"}, {"label": "P-I", "prob": "0.9999994", "token": "are"}, {"label": "P-I", "prob": "0.9999995", "token": "ruthless"}, {"label": "P-I", "prob": "0.9999994", "token": "the"}, {"label": "P-I", "prob": "0.9999995", "token": "more"}, {"label": "P-I", "prob": "0.99999964", "token": "likely"}, {"label": "P-I", "prob": "0.9999995", "token": "we"}, {"label": "P-I", "prob": "0.99999964", "token": "are"}, {"label": "P-I", "prob": "0.99999964", "token": "to"}, {"label": "P-I", "prob": "0.9999994", "token": "be"}, {"label": "P-I", "prob": "0.9999995", "token": "in"}, {"label": "P-I", "prob": "0.9999994", "token": "private"}, {"label": "O", "prob": "0.99999976", "token": "."}], [{"label": "O", "prob": "0.99998796", "token": "I"}, {"label": "P-B", "prob": "0.9999677", "token": "mean"}, {"label": "P-I", "prob": "0.9999908", "token": "Ella"}, {"label": "P-I", "prob": "0.9999951", "token": "is"}, {"label": "P-I", "prob": "0.9999987", "token": "absolutely"}, {"label": "P-I", "prob": "0.9999995", "token": "right"}, {"label": "P-I", "prob": "0.99999964", "token": "politicians"}, {"label": "P-I", "prob": "0.9999993", "token": "should"}, {"label": "P-I", "prob": "0.9999995", "token": "not"}, {"label": "P-I", "prob": "0.9999995", "token": "be"}, {"label": "P-I", "prob": "0.99999964", "token": "moral"}, {"label": "P-I", "prob": "0.9999995", "token": "figureheads"}, {"label": "P-I", "prob": "0.99999964", "token": "but"}, {"label": "P-I", "prob": "0.9999995", "token": "equally"}, {"label": "P-I", "prob": "0.9999995", "token": "they"}, {"label": "P-I", "prob": "0.9999989", "token": "must"}, {"label": "P-I", "prob": "0.9999994", "token": "not"}, {"label": "P-I", "prob": "0.9999995", "token": "encourage"}, {"label": "P-I", "prob": "0.99999964", "token": "degeneracy"}, {"label": "O", "prob": "0.99999976", "token": "."}], [{"label": "P-B", "prob": "0.9997775", "token": "Giles"}, {"label": "P-I", "prob": "0.9999832", "token": "Fraser"}, {"label": "O", "prob": "0.6687364", "token": ":"}, {"label": "P-I", "prob": "0.99999857", "token": "Ruthlessness"}, {"label": "P-I", "prob": "0.9999993", "token": "is"}, {"label": "P-I", "prob": "0.9999993", "token": "a"}, {"label": "P-I", "prob": "0.9999994", "token": "virtue"}, {"label": "P-I", "prob": "0.9999994", "token": "with"}, {"label": "P-I", "prob": "0.99999964", "token": "regard"}, {"label": "P-I", "prob": "0.99999964", "token": "to"}, {"label": "P-I", "prob": "0.9999994", "token": "having"}, {"label": "P-I", "prob": "0.9999995", "token": "to"}, {"label": "P-I", "prob": "0.9999994", "token": "get"}, {"label": "P-I", "prob": "0.9999994", "token": "things"}, {"label": "P-I", "prob": "0.9999993", "token": "done"}, {"label": "P-I", "prob": "0.99999917", "token": "sometimes"}, {"label": "P-I", "prob": "0.9999988", "token": "and"}, {"label": "P-I", "prob": "0.9999994", "token": "thats"}, {"label": "P-I", "prob": "0.9999995", "token": "the"}, {"label": "P-I", "prob": "0.9999993", "token": "nature"}, {"label": "P-I", "prob": "0.9999993", "token": "of"}, {"label": "P-I", "prob": "0.9999994", "token": "..."}, {"label": "P-I", "prob": "0.99999964", "token": "Mona"}, {"label": "P-I", "prob": "0.9999994", "token": "Siddiqui"}, {"label": "P-I", "prob": "0.9999912", "token": ":"}, {"label": "P-I", "prob": "0.9999993", "token": "Irrespective"}, {"label": "P-I", "prob": "0.99999964", "token": "of"}, {"label": "P-I", "prob": "0.99999964", "token": "the"}, {"label": "P-I", "prob": "0.9999995", "token": "fallout"}, {"label": "P-I", "prob": "0.9999995", "token": "irrespective"}, {"label": "P-I", "prob": "0.9999994", "token": "of"}, {"label": "P-I", "prob": "0.9999993", "token": "the"}, {"label": "P-I", "prob": "0.9999994", "token": "consequences"}, {"label": "O", "prob": "0.99999976", "token": "."}], [{"label": "P-B", "prob": "0.9997687", "token": "Giles"}, {"label": "P-I", "prob": "0.99997926", "token": "Fraser"}, {"label": "O", "prob": "0.7243656", "token": ":"}, {"label": "P-I", "prob": "0.9999944", "token": "Not"}, {"label": "P-I", "prob": "0.9999994", "token": "always"}, {"label": "P-I", "prob": "0.9999995", "token": "it"}, {"label": "P-I", "prob": "0.99999917", "token": "has"}, {"label": "P-I", "prob": "0.99999964", "token": "to"}, {"label": "P-I", "prob": "0.9999994", "token": "be"}, {"label": "P-I", "prob": "0.99999976", "token": "balanced"}, {"label": "P-I", "prob": "0.9999995", "token": "with"}, {"label": "P-I", "prob": "0.9999994", "token": "other"}, {"label": "P-I", "prob": "0.99999976", "token": "things"}, {"label": "P-I", "prob": "0.99999964", "token": "but"}, {"label": "P-I", "prob": "0.9999995", "token": "its"}, {"label": "P-I", "prob": "0.99999964", "token": "nonetheless"}, {"label": "P-I", "prob": "0.9999995", "token": "still"}, {"label": "P-I", "prob": "0.99999964", "token": "a"}, {"label": "P-I", "prob": "0.9999995", "token": "virtue"}, {"label": "P-I", "prob": "0.9999994", "token": "its"}, {"label": "P-I", "prob": "0.9999995", "token": "something"}, {"label": "P-I", "prob": "0.9999993", "token": "that"}, {"label": "P-I", "prob": "0.9999993", "token": "you"}, {"label": "P-I", "prob": "0.99999917", "token": "couldnt"}, {"label": "P-I", "prob": "0.99999917", "token": "be"}, {"label": "P-I", "prob": "0.9999994", "token": "a"}, {"label": "P-I", "prob": "0.9999994", "token": "politician"}, {"label": "P-I", "prob": "0.99999917", "token": "without"}, {"label": "O", "prob": "0.99999976", "token": "."}]]

In [45]:
 print(get_segments(result2))

{'Tim Stanley : Giles name for me please one thing that in your private life is utterly immoral but in politics would be regarded as a virtue ?': 'premise', 'Giles Fraser Well he said that ruthlessness was a classic example and thats what he focused on': 'premise', 'So ruthlessness is not something that is a domestic virtue but it may well be a public one because there are times in which you have to do things that youre just': 'premise', 'Tim Stanley : But the public and the private are not separate': 'premise', 'more the public people are ruthless the more likely we are to be in private': 'premise', 'mean Ella is absolutely right politicians should not be moral figureheads but equally they must not encourage degeneracy': 'premise', 'Giles Fraser Ruthlessness is a virtue with regard to having to get things done sometimes and thats the nature of ... Mona Siddiqui : Irrespective of the fallout irrespective of the consequences': 'premise', 'Giles Fraser Not always it has to be balanced wi

In [ ]:
Tim Stanley:  Giles name for me please one thing that in your private life is utterly immoral but in politics would be regarded as a virtue?
    
Giles Fraser:  Well he said that ruthlessness was a classic example and thats what he focused on.   
So ruthlessness is not something that is a domestic virtue but it may well be a public one because there are times    in which you have to do things that youre just.  

Tim Stanley:  But the public and the private are not separate. 

The more the public people are ruthless the more likely we are to be in private.   
I mean Ella is absolutely right politicians should not be moral figureheads but equally they must not encourage degeneracy. 

Giles Fraser:  Ruthlessness is a virtue with regard to having to get things done sometimes and thats the nature of...   
Mona Siddiqui:  Irrespective of the fallout irrespective of the consequences. 
Giles Fraser:  Not always it has to be balanced with other things but its nonetheless still a virtue its something that you couldnt be a politician without.   
   


In [46]:
prop1, prop2 = "pp", "pp2"
p1p2=f'{prop1} {prop2}'



In [47]:
p1p2

'pp pp2'

In [60]:
def get_next_max_id(nodes, n_type):
    """
    This function takes a list of nodes and returns the maximum node ID.

    Arguments:
    - nodes (List[Dict]): a list of nodes, where each node is a dictionary containing a node ID

    Returns:
    - (int): the maximum node ID in the list of nodes
    """
    # Initialize a variable to store the maximum node ID found so far
    max_id  = 0
    lef_n_id, right_n_id = 0, ""
   
    if isinstance(nodes[0][n_type],str):
      if "_" in nodes[0][n_type]:
          
          #logging.debug('with hyphen')       
          # Loop through each node in the list of nodes
          for node in nodes:
              # Check if the current node ID is greater than the current maximum
              #logging.debug(node)
              temp_id = node[n_type]
              if "_" in temp_id:
                  nodeid_parsed = temp_id.split("_")
                  lef_n_id, right_n_id = int(nodeid_parsed[0]), nodeid_parsed[1]
                  if lef_n_id > max_id:
                      max_id = lef_n_id
          #logging.debug(str(int(max_id)+1)+"_"+str(right_n_id))
          return str(int(max_id)+1)+"_"+str(right_n_id)
      else:
        for node in nodes:
            # Check if the current node ID is greater than the current maximum
            temp_id = int(node[n_type])     
            if temp_id > max_id:
                # If it is, update the maximum to the current node ID
                max_id = temp_id   
        # Return the maximum node ID found
        return str(max_id+1)

    elif isinstance(nodes[0][n_type],int):	
      for node in nodes:
          # Check if the current node ID is greater than the current maximum
          temp_id = node[n_type]     
          if temp_id > max_id:
              # If it is, update the maximum to the current node ID
              max_id = temp_id   
      # Return the maximum node ID found
      return max_id+1

In [51]:
null = "None"
false = False
true = True
data = {   "AIF": {      
    "nodes": [ 
        {
                "nodeID": "2_164761847968435155",
                "text":"If the main Scottish elections return a full independence government, what moral and legalright does the Westminster government have to deny another independence referendum","type":"L"
            },
            {
                    "nodeID": "3_164761847968435155",
                    "text":"Blair Unknown : If the main Scottish elections return a full independence government, what moral and legal right does the Westminster government have to deny another independence referendum","type":"L"
                },
                {
                        "nodeID": "4_164761847968435155",
                        "text": "Pure Questioning",
                        "type": "YA"
                    },
                    {                        "nodeID": "5_164761847968435155",
                        "text":"If the main Scottish elections return a full independence government, Westminster government have xxx moral and legal right to deny another independence referendum","type":"I"
                    },
                    
                    {
                            "nodeID": "7_164761847968435155",
                            "text":"Kamila: Blair Unknown : If the main Scottish elections return a full independence government, what moral and legal right does the Westminster government have to deny another independence referendum","type":"L"
                        },
                        
                        {
                                "nodeID": "8_164761847968435155",
                                "text": "Douglas Ross : I want to stop a pro-independence majority government",
                                "type": "L"
                            },
                            {
                                "nodeID": "9_164761847968435155",
                                "text": "Asserting",
                                "type": "YA"
                            },
                            {
                                "nodeID": "10_164761847968435155",
                                "text": "Douglas Ross wants to stop a pro-independence majority government",
                                "type": "I"
                            },
                            {
                                "nodeID": "12_164761847968435155",
                                "text": "Kamila: Douglas Ross : I want to stop a pro-independence majority government",
                                "type": "L"
                            },
                            {
                                "nodeID": "14_164761847968435155",
                                "text": "Default Transition",
                                "type": "TA"
                            },
                            {
                                "nodeID": "15_164761847968435155",
                                "text": "Douglas Ross : I want to stop the SNP having a majority at this election, as the Scottish Conservatives did five years ago",
                                "type": "L"
                            },
                            {
                                "nodeID": "16_164761847968435155",
                                "text": "Asserting",
                                "type": "YA"
                            },
                            {
                                "nodeID": "17_164761847968435155",
                                "text": "Douglas Ross wants to stop the SNP having a majority at this election, as the Scottish Conservatives did five years ago",
                                "type": "I"
                            },
                            {
                                "nodeID": "19_164761847968435155",
                                "text": "Kamila: Douglas Ross : I want to stop the SNP having a majority at this election, as the Scottish Conservatives did five years ago",
                                "type": "L"
                            },
                            {
                                "nodeID": "20_164761847968435155",
                                "text": "Douglas Ross : It was by people uniting behind the Scottish Conservatives that stopped that SNP majority in 2016",
                                "type": "L"
                            },
                            {
                                "nodeID": "21_164761847968435155",
                                "text": "Asserting",
                                "type": "YA"
                            },
                            {
                                "nodeID": "22_164761847968435155",
                                "text": "It was by people uniting behind the Scottish Conservatives that stopped that SNP majority in 2016",
                                "type": "I"
                            },
                            {
                                "nodeID": "24_164761847968435155",
                                "text": "Kamila: Douglas Ross : It was by people uniting behind the Scottish Conservatives that stopped that SNP majority in 2016",
                                "type": "L"
                            },
                            {
                                "nodeID": "26_164761847968435155",
                                "text": "Default Transition",
                                "type": "TA"
                            },
                            {
                                "nodeID": "28_164761847968435155",
                                "text": "Default Rephrase",
                                "type": "MA"
                            },
                            {
                                "nodeID": "30_164761847968435155",
                                "text": "Restating",
                                "type": "YA"
                            },
                            {
                                "nodeID": "32_164761847968435155",
                                "text": "Default Transition",
                                "type": "TA"
                            },
                            {
                                "nodeID": "33_164761847968435155",
                                "text": "Douglas Ross : people can do that again at this election",
                                "type": "L"
                            },
                            {
                                "nodeID": "34_164761847968435155",
                                "text": "Asserting",
                                "type": "YA"
                            },
                            {
                                "nodeID": "35_164761847968435155",
                                "text": "people can unite again at this election to stop the SNP majority",
                                "type": "I"
                            },
                            {
                                "nodeID": "37_164761847968435155",
                                "text": "Kamila: Douglas Ross : people can do that again at this election",
                                "type": "L"
                            },
                            {
                                "nodeID": "39_164761847968435155",
                                "text": "Default Transition",
                                "type": "TA"
                            },
                            {
                                "nodeID": "41_164761847968435155",
                                "text": "Default Inference",
                                "type": "RA"
                            },
                            {
                                "nodeID": "43_164761847968435155",
                                "text": "Arguing",
                                "type": "YA"
                            }
                        ],
                        "edges": [
                            {
                                "edgeID": 1,
                                "fromID": "4_164761847968435155",
                                "toID": "5_164761847968435155"
                            },
                            {
                                "edgeID": 2,
                                "fromID": "3_164761847968435155",
                                "toID": "4_164761847968435155"
                            },
                            {
                                "edgeID": 3,
                                "fromID": "9_164761847968435155",
                                "toID": "10_164761847968435155"
                            },
                            {
                                "edgeID": 4,
                                "fromID": "8_164761847968435155",
                                "toID": "9_164761847968435155"
                            },
                            {
                                "edgeID": 5,
                                "fromID": "3_164761847968435155",
                                "toID": "14_164761847968435155"
                            },
                            {
                                "edgeID": 6,
                                "fromID": "14_164761847968435155",
                                "toID": "8_164761847968435155"
                            },
                            {
                                "edgeID": 7,
                                "fromID": "16_164761847968435155",
                                "toID": "17_164761847968435155"
                            },
                            {
                                "edgeID": 8,
                                "fromID": "15_164761847968435155",
                                "toID": "16_164761847968435155"
                            },
                            {
                                "edgeID": 9,
                                "fromID": "21_164761847968435155",
                                "toID": "22_164761847968435155"
                            },
                            {
                                "edgeID": 10,
                                "fromID": "20_164761847968435155",
                                "toID": "21_164761847968435155"
                            },
                            {
                                "edgeID": 11,
                                "fromID": "8_164761847968435155",
                                "toID": "26_164761847968435155"
                            },
                            {
                                "edgeID": 12,
                                "fromID": "26_164761847968435155",
                                "toID": "15_164761847968435155"
                            },
                            {
                                "edgeID": 13,
                                "fromID": "17_164761847968435155",
                                "toID": "28_164761847968435155"
                            },
                            {
                                "edgeID": 14,
                                "fromID": "28_164761847968435155",
                                "toID": "10_164761847968435155"
                            },
                            {
                                "edgeID": 15,
                                "fromID": "26_164761847968435155",
                                "toID": "30_164761847968435155"
                            },
                            {
                                "edgeID": 16,
                                "fromID": "30_164761847968435155",
                                "toID": "28_164761847968435155"
                            },
                            {
                                "edgeID": 17,
                                "fromID": "15_164761847968435155",
                                "toID": "32_164761847968435155"
                            },
                            {
                                "edgeID": 18,
                                "fromID": "32_164761847968435155",
                                "toID": "20_164761847968435155"
                            },
                            {
                                "edgeID": 19,
                                "fromID": "34_164761847968435155",
                                "toID": "35_164761847968435155"
                            },
                            {
                                "edgeID": 20,
                                "fromID": "33_164761847968435155",
                                "toID": "34_164761847968435155"
                            },
                            {
                                "edgeID": 21,
                                "fromID": "20_164761847968435155",
                                "toID": "39_164761847968435155"
                            },
                            {
                                "edgeID": 22,
                                "fromID": "39_164761847968435155",
                                "toID": "33_164761847968435155"
                            },
                            {
                                "edgeID": 23,
                                "fromID": "22_164761847968435155",
                                "toID": "41_164761847968435155"
                            },
                            {
                                "edgeID": 24,
                                "fromID": "41_164761847968435155",
                                "toID": "35_164761847968435155"
                            },
                            {
                                "edgeID": 25,
                                "fromID": "39_164761847968435155",
                                "toID": "43_164761847968435155"
                            },
                            {
                                "edgeID": 26,
                                "fromID": "43_164761847968435155",
                                "toID": "41_164761847968435155"
                            }
                        ],
                        "schemefulfillments": [
                            {
                                "nodeID": "2_164761847968435155",
                                "schemeID": "0"
                            },
                            {
                                "nodeID": "3_164761847968435155",
                                "schemeID": "0"
                            },
                            {
                                "nodeID": "4_164761847968435155",
                                "schemeID": "91"
                            },
                            {
                                "nodeID": "5_164761847968435155",
                                "schemeID": "0"
                            },
                            {
                                "nodeID": "7_164761847968435155",
                                "schemeID": "0"
                            },
                            {
                                "nodeID": "8_164761847968435155",
                                "schemeID": "0"
                            },
                            {
                                "nodeID": "9_164761847968435155",
                                "schemeID": "74"
                            },
                            {
                                "nodeID": "10_164761847968435155",
                                "schemeID": "0"
                            },
                            {
                                "nodeID": "12_164761847968435155",
                                "schemeID": "0"
                            },
                            {
                                "nodeID": "14_164761847968435155",
                                "schemeID": "82"
                            },
                            {
                                "nodeID": "15_164761847968435155",
                                "schemeID": "0"
                            },
                            {
                                "nodeID": "16_164761847968435155",
                                "schemeID": "74"
                            },
                            {
                                "nodeID": "17_164761847968435155",
                                "schemeID": "0"
                            },
                            {
                                "nodeID": "19_164761847968435155",
                                "schemeID": "0"
                            },
                            {
                                "nodeID": "20_164761847968435155",
                                "schemeID": "0"
                            },
                            {
                                "nodeID": "21_164761847968435155",
                                "schemeID": "74"
                            },
                            {
                                "nodeID": "22_164761847968435155",
                                "schemeID": "0"
                            },
                            {
                                "nodeID": "24_164761847968435155",
                                "schemeID": "0"
                            },
                            {
                                "nodeID": "26_164761847968435155",
                                "schemeID": "82"
                            },
                            {
                                "nodeID": "28_164761847968435155",
                                "schemeID": "144"
                            },
                            {
                                "nodeID": "30_164761847968435155",
                                "schemeID": "101"
                            },
                            {
                                "nodeID": "32_164761847968435155",
                                "schemeID": "82"
                            },
                            {
                                "nodeID": "33_164761847968435155",
                                "schemeID": "0"
                            },
                            {
                                "nodeID": "34_164761847968435155",
                                "schemeID": "74"
                            },
                            {
                                "nodeID": "35_164761847968435155",
                                "schemeID": "0"
                            },
                            {
                                "nodeID": "37_164761847968435155",
                                "schemeID": "0"
                            },
                            {
                                "nodeID": "39_164761847968435155",
                                "schemeID": "82"
                            },
                            {
                                "nodeID": "41_164761847968435155",
                                "schemeID": "72"
                            },
                            {
                                "nodeID": "43_164761847968435155",
                                "schemeID": "80"
                            }
                        ],
                        "descriptorfulfillments": [],
                        "participants": [
                            {
                                "participantID": 1,
                                "firstname": "Participant",
                                "surname": "Rachel"
                            },
                            {
                                "participantID": 2,
                                "firstname": "Participant",
                                "surname": "Carolyn"
                            },
                            {
                                "participantID": 3,
                                "firstname": "Participant",
                                "surname": "Mo"
                            },
                            {
                                "participantID": 4,
                                "firstname": "Participant",
                                "surname": "Zoe"
                            },
                            {
                                "participantID": 5,
                                "firstname": "Participant",
                                "surname": "Claire"
                            },
                            {
                                "participantID": 6,
                                "firstname": "Participant",
                                "surname": "Lee"
                            },
                            {
                                "participantID": 7,
                                "firstname": "Participant",
                                "surname": "Sally"
                            },
                            {
                                "participantID": 8,
                                "firstname": "Participant",
                                "surname": "Kalbir"
                            },
                            {
                                "participantID": 9,
                                "firstname": "Participant",
                                "surname": "Alice"
                            },
                            {
                                "participantID": 10,
                                "firstname": "Anne",
                                "surname": "Robinson"
                            },
                            {
                                "participantID": 11,
                                "firstname": "Diane",
                                "surname": "Munday"
                            },
                            {
                                "participantID": 12,
                                "firstname": "Neena",
                                "surname": "Modi"
                            },
                            {
                                "participantID": 13,
                                "firstname": "David",
                                "surname": "Steel"
                            },
                            {
                                "participantID": 14,
                                "firstname": "Neil",
                                "surname": "Lyndon"
                            },
                            {
                                "participantID": 15,
                                "firstname": "Fiona",
                                "surname": "Bruce"
                            },
                            {
                                "participantID": 16,
                                "firstname": "Anas",
                                "surname": "Sarwar"
                            },
                            {
                                "participantID": 17,
                                "firstname": "Douglas",
                                "surname": "Ross"
                            },
                            {
                                "participantID": 18,
                                "firstname": "Keith",
                                "surname": "Brown"
                            },
                            {
                                "participantID": 19,
                                "firstname": "Lorna",
                                "surname": "Slater"
                            },
                            {
                                "participantID": 20,
                                "firstname": "Willie",
                                "surname": "Renner"
                            },
                            {
                                "participantID": 21,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT01"
                            },
                            {
                                "participantID": 22,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT02"
                            },
                            {
                                "participantID": 23,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT03"
                            },
                            {
                                "participantID": 24,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT04"
                            },
                            {
                                "participantID": 25,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT05"
                            },
                            {
                                "participantID": 26,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT06"
                            },
                            {
                                "participantID": 27,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT07"
                            },
                            {
                                "participantID": 28,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT08"
                            },
                            {
                                "participantID": 29,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT09"
                            },
                            {
                                "participantID": 30,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT10"
                            },
                            {
                                "participantID": 31,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT11"
                            },
                            {
                                "participantID": 32,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT12"
                            },
                            {
                                "participantID": 33,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT13"
                            },
                            {
                                "participantID": 34,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT14"
                            },
                            {
                                "participantID": 35,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT15"
                            },
                            {
                                "participantID": 36,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT16"
                            },
                            {
                                "participantID": 37,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT17"
                            },
                            {
                                "participantID": 38,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT18"
                            },
                            {
                                "participantID": 39,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT19"
                            },
                            {
                                "participantID": 40,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT20"
                            },
                            {
                                "participantID": 41,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT21"
                            },
                            {
                                "participantID": 42,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT22"
                            },
                            {
                                "participantID": 43,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT23"
                            },
                            {
                                "participantID": 44,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT24"
                            },
                            {
                                "participantID": 45,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT25"
                            },
                            {
                                "participantID": 46,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT26"
                            },
                            {
                                "participantID": 47,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT27"
                            },
                            {
                                "participantID": 48,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT28"
                            },
                            {
                                "participantID": 49,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT29"
                            },
                            {
                                "participantID": 50,
                                "firstname": "AudienceMember",
                                "surname": "20210429QT30"
                            },
                            {
                                "participantID": 51,
                                "firstname": "Blair",
                                "surname": "Unknown"
                            }
                        ],
                        "locutions": [
                            {
                                "nodeID": "3_164761847968435155",
                                "personID": 51,
                                "start": "2020-10-15 21:51:25",
                                "end": null
                            },
                            {
                                "nodeID": "8_164761847968435155",
                                "personID": 17,
                                "start": "2020-10-15 21:51:33",
                                "end": null
                            },
                            {
                                "nodeID": "15_164761847968435155",
                                "personID": 17,
                                "start": "2020-10-15 21:51:39",
                                "end": null
                            },
                            {
                                "nodeID": "20_164761847968435155",
                                "personID": 17,
                                "start": "2020-10-15 21:51:50",
                                "end": null
                            },
                            {
                                "nodeID": "33_164761847968435155",
                                "personID": 17,
                                "start": "2020-10-15 21:52:00",
                                "end": null
                            }
                        ]
                    },
                    "OVA": {
                        "firstname": "K",
                        "surname": "G",
                        "nodes": [
                            {
                                "nodeID": "2_164761847968435155",
                                "visible": false,
                                "x": 755,
                                "y": 153,
                                "timestamp": "Thu Oct 15 2020 22:51:25 GMT+0100 (British Summer Time)"
                            },
                            {
                                "nodeID": "3_164761847968435155",
                                "visible": true,
                                "x": 747,
                                "y": 111,
                                "timestamp": "Thu Oct 15 2020 22:51:25 GMT+0100 (British Summer Time)"
                            },
                            {
                                "nodeID": "4_164761847968435155",
                                "visible": true,
                                "x": 560,
                                "y": 166,
                                "timestamp": ""
                            },
                            {
                                "nodeID": "5_164761847968435155",
                                "visible": true,
                                "x": 271,
                                "y": 119,
                                "timestamp": ""
                            },
                            {
                                "nodeID": "7_164761847968435155",
                                "visible": false,
                                "x": 748,
                                "y": -50,
                                "timestamp": ""
                            },
                            {
                                "nodeID": "8_164761847968435155",
                                "visible": true,
                                "x": 759,
                                "y": 340,
                                "timestamp": "Thu Oct 15 2020 22:51:33 GMT+0100 (British Summer Time)"
                            },
                            {
                                "nodeID": "9_164761847968435155",
                                "visible": true,
                                "x": 559,
                                "y": 340,
                                "timestamp": ""
                            },
                            {
                                "nodeID": "10_164761847968435155",
                                "visible": true,
                                "x": 283,
                                "y": 340,
                                "timestamp": ""
                            },
                            {
                                "nodeID": "12_164761847968435155",
                                "visible": false,
                                "x": 759,
                                "y": -50,
                                "timestamp": ""
                            },
                            {
                                "nodeID": "14_164761847968435155",
                                "visible": true,
                                "x": 786,
                                "y": 269,
                                "timestamp": ""
                            },
                            {
                                "nodeID": "15_164761847968435155",
                                "visible": true,
                                "x": 762,
                                "y": 491,
                                "timestamp": "Thu Oct 15 2020 22:51:39 GMT+0100 (British Summer Time)"
                            },
                            {
                                "nodeID": "16_164761847968435155",
                                "visible": true,
                                "x": 582,
                                "y": 506,
                                "timestamp": ""
                            },
                            {
                                "nodeID": "17_164761847968435155",
                                "visible": true,
                                "x": 290,
                                "y": 492,
                                "timestamp": ""
                            },
                            {
                                "nodeID": "19_164761847968435155",
                                "visible": false,
                                "x": 762,
                                "y": -50,
                                "timestamp": ""
                            },
                            {
                                "nodeID": "20_164761847968435155",
                                "visible": true,
                                "x": 765,
                                "y": 659,
                                "timestamp": "Thu Oct 15 2020 22:51:50 GMT+0100 (British Summer Time)"
                            },
                            {
                                "nodeID": "21_164761847968435155",
                                "visible": true,
                                "x": 565,
                                "y": 678,
                                "timestamp": ""
                            },
                            {
                                "nodeID": "22_164761847968435155",
                                "visible": true,
                                "x": 290,
                                "y": 675,
                                "timestamp": ""
                            },
                            {
                                "nodeID": "24_164761847968435155",
                                "visible": false,
                                "x": 765,
                                "y": -50,
                                "timestamp": ""
                            },
                            {
                                "nodeID": "26_164761847968435155",
                                "visible": true,
                                "x": 786.5,
                                "y": 428,
                                "timestamp": ""
                            },
                            {
                                "nodeID": "28_164761847968435155",
                                "visible": true,
                                "x": 313.5,
                                "y": 433,
                                "timestamp": ""
                            },
                            {
                                "nodeID": "30_164761847968435155",
                                "visible": true,
                                "x": 555,
                                "y": 435,
                                "timestamp": ""
                            },
                            {
                                "nodeID": "32_164761847968435155",
                                "visible": true,
                                "x": 810.5,
                                "y": 605,
                                "timestamp": ""
                            },
                            {
                                "nodeID": "33_164761847968435155",
                                "visible": true,
                                "x": 765,
                                "y": 827,
                                "timestamp": "Thu Oct 15 2020 22:52:00 GMT+0100 (British Summer Time)"
                            },
                            {
                                "nodeID": "34_164761847968435155",
                                "visible": true,
                                "x": 565,
                                "y": 827,
                                "timestamp": ""
                            },
                            {
                                "nodeID": "35_164761847968435155",
                                "visible": true,
                                "x": 292,
                                "y": 831,
                                "timestamp": ""
                            },
                            {
                                "nodeID": "37_164761847968435155",
                                "visible": false,
                                "x": 765,
                                "y": -50,
                                "timestamp": ""
                            },
                            {
                                "nodeID": "39_164761847968435155",
                                "visible": true,
                                "x": 787,
                                "y": 766,
                                "timestamp": ""
                            },
                            {
                                "nodeID": "41_164761847968435155",
                                "visible": true,
                                "x": 319,
                                "y": 764,
                                "timestamp": ""
                            },
                            {
                                "nodeID": "43_164761847968435155",
                                "visible": true,
                                "x": 558,
                                "y": 770,
                                "timestamp": ""
                            }
                        ],
                        "edges": [
                            {
                                "fromID": "4_164761847968435155",
                                "toID": "5_164761847968435155",
                                "visible": true
                            },
                            {
                                "fromID": "3_164761847968435155",
                                "toID": "4_164761847968435155",
                                "visible": true
                            },
                            {
                                "fromID": "9_164761847968435155",
                                "toID": "10_164761847968435155",
                                "visible": true
                            },
                            {
                                "fromID": "8_164761847968435155",
                                "toID": "9_164761847968435155",
                                "visible": true
                            },
                            {
                                "fromID": "3_164761847968435155",
                                "toID": "14_164761847968435155",
                                "visible": true
                            },
                            {
                                "fromID": "14_164761847968435155",
                                "toID": "8_164761847968435155",
                                "visible": true
                            },
                            {
                                "fromID": "16_164761847968435155",
                                "toID": "17_164761847968435155",
                                "visible": true
                            },
                            {
                                "fromID": "15_164761847968435155",
                                "toID": "16_164761847968435155",
                                "visible": true
                            },
                            {
                                "fromID": "21_164761847968435155",
                                "toID": "22_164761847968435155",
                                "visible": true
                            },
                            {
                                "fromID": "20_164761847968435155",
                                "toID": "21_164761847968435155",
                                "visible": true
                            },
                            {
                                "fromID": "8_164761847968435155",
                                "toID": "26_164761847968435155",
                                "visible": true
                            },
                            {
                                "fromID": "26_164761847968435155",
                                "toID": "15_164761847968435155",
                                "visible": true
                            },
                            {
                                "fromID": "17_164761847968435155",
                                "toID": "28_164761847968435155",
                                "visible": true
                            },
                            {
                                "fromID": "28_164761847968435155",
                                "toID": "10_164761847968435155",
                                "visible": true
                            },
                            {
                                "fromID": "26_164761847968435155",
                                "toID": "30_164761847968435155",
                                "visible": true
                            },
                            {
                                "fromID": "30_164761847968435155",
                                "toID": "28_164761847968435155",
                                "visible": true
                            },
                            {
                                "fromID": "15_164761847968435155",
                                "toID": "32_164761847968435155",
                                "visible": true
                            },
                            {
                                "fromID": "32_164761847968435155",
                                "toID": "20_164761847968435155",
                                "visible": true
                            },
                            {
                                "fromID": "34_164761847968435155",
                                "toID": "35_164761847968435155",
                                "visible": true
                            },
                            {
                                "fromID": "33_164761847968435155",
                                "toID": "34_164761847968435155",
                                "visible": true
                            },
                            {
                                "fromID": "20_164761847968435155",
                                "toID": "39_164761847968435155",
                                "visible": true
                            },
                            {
                                "fromID": "39_164761847968435155",
                                "toID": "33_164761847968435155",
                                "visible": true
                            },
                            {
                                "fromID": "22_164761847968435155",
                                "toID": "41_164761847968435155",
                                "visible": true
                            },
                            {
                                "fromID": "41_164761847968435155",
                                "toID": "35_164761847968435155",
                                "visible": true
                            },
                            {
                                "fromID": "39_164761847968435155",
                                "toID": "43_164761847968435155",
                                "visible": true
                            },
                            {
                                "fromID": "43_164761847968435155",
                                "toID": "41_164761847968435155",
                                "visible": true
                            }
                        ]
                    },
                    "text": {
                        "txt": "Fiona Bruce[00:01:10] Our first question tonight is from Blair Wondin.<div><br></div><div>Blair Unknown[00:01:22] Hello. <span class=\"highlighted\" id=\"node1\"><span class=\"highlighted\" id=\"node3_164761847968435155\">If the main Scottish elections return a full independence government, what moral and legal right does the Westminster government have to deny another independence referendum</span></span>?</div><div><br></div><div>Fiona Bruce[00:01:32] Doug?</div><div><br></div><div>Douglas Ross[00:01:33] Well, <span class=\"highlighted\" id=\"node8_164761847968435155\">I want to stop a pro-independence majority government</span>, <span class=\"highlighted\" id=\"node15_164761847968435155\">I want to stop the SNP having a majority at this election, as the Scottish Conservatives did five years ago</span>. <span class=\"highlighted\" id=\"node20_164761847968435155\">It was by people uniting behind the Scottish Conservatives that stopped that SNP majority in 2016</span>, and <span class=\"highlighted\" id=\"node33_164761847968435155\">people can do that again at this election</span>. </div>",
                        "url": ""
                    }
                }

In [52]:
nodes = data['AIF']['nodes']
edges = data['AIF']['edges']

In [62]:
print(get_next_max_id(edges, 'edgeID'))

27


In [59]:
edges

[{'edgeID': 1,
  'fromID': '4_164761847968435155',
  'toID': '5_164761847968435155'},
 {'edgeID': 2,
  'fromID': '3_164761847968435155',
  'toID': '4_164761847968435155'},
 {'edgeID': 3,
  'fromID': '9_164761847968435155',
  'toID': '10_164761847968435155'},
 {'edgeID': 4,
  'fromID': '8_164761847968435155',
  'toID': '9_164761847968435155'},
 {'edgeID': 5,
  'fromID': '3_164761847968435155',
  'toID': '14_164761847968435155'},
 {'edgeID': 6,
  'fromID': '14_164761847968435155',
  'toID': '8_164761847968435155'},
 {'edgeID': 7,
  'fromID': '16_164761847968435155',
  'toID': '17_164761847968435155'},
 {'edgeID': 8,
  'fromID': '15_164761847968435155',
  'toID': '16_164761847968435155'},
 {'edgeID': 9,
  'fromID': '21_164761847968435155',
  'toID': '22_164761847968435155'},
 {'edgeID': 10,
  'fromID': '20_164761847968435155',
  'toID': '21_164761847968435155'},
 {'edgeID': 11,
  'fromID': '8_164761847968435155',
  'toID': '26_164761847968435155'},
 {'edgeID': 12,
  'fromID': '26_16476184

In [66]:
data3 = {"AIF": {"descriptorfulfillments": [], "edges": [{"edgeID": 5, "fromID": 1, "toID": 4}, {"edgeID": 6, "fromID": 4, "toID": 3}, {"edgeID": 13, "fromID": 9, "toID": 12}, {"edgeID": 14, "fromID": 12, "toID": 11}], "locutions": [{"nodeID": 1, "personID": "Stanley:"}, {"nodeID": 9, "personID": "Fraser:"}], "nodes": [{"nodeID": 1, "text": "  Giles name for me please one thing that in your private life is utterly immoral but in politics would be regarded as a virtue? ", "type": "L"}, {"nodeID": 3, "text": "  Giles name for me please one thing that in your private life is utterly immoral but in politics would be regarded as a virtue? ", "type": "I"}, {"nodeID": 4, "text": "Default Illocuting", "type": "YA"}, {"nodeID": 9, "text": "  Well he said that ruthlessness was a classic example and thats what he focused on.   So ruthlessness is not something that is a domestic virtue know...", "type": "L"}, {"nodeID": 11, "text": "  Well he said that ruthlessness was a classic example and thats what he focused on.   So ruthlessness is not something that is a domestic virtue know...", "type": "I"}, {"nodeID": 12, "text": "Default Illocuting", "type": "YA"}], "schemefulfillments": []}, "OVA": {}, "text": " Stanley: <span class=\"highlighted\" id=\"0\">  Giles name for me please one thing that in your private life is utterly immoral but in politics would be regarded as a virtue? </span>.<br><br> Fraser: <span class=\"highlighted\" id=\"1\">  Well he said that ruthlessness was a classic example and thats what he focused on.   So ruthlessness is not something that is a domestic virtue know...</span>.<br><br>"}
nodes = data3['AIF']['nodes']
edges = data3['AIF']['edges']

In [71]:
print(get_next_max_id(nodes, 'nodeID'))

13


In [72]:
nodes

[{'nodeID': 1,
  'text': '  Giles name for me please one thing that in your private life is utterly immoral but in politics would be regarded as a virtue? ',
  'type': 'L'},
 {'nodeID': 3,
  'text': '  Giles name for me please one thing that in your private life is utterly immoral but in politics would be regarded as a virtue? ',
  'type': 'I'},
 {'nodeID': 4, 'text': 'Default Illocuting', 'type': 'YA'},
 {'nodeID': 9,
  'text': '  Well he said that ruthlessness was a classic example and thats what he focused on.   So ruthlessness is not something that is a domestic virtue know...',
  'type': 'L'},
 {'nodeID': 11,
  'text': '  Well he said that ruthlessness was a classic example and thats what he focused on.   So ruthlessness is not something that is a domestic virtue know...',
  'type': 'I'},
 {'nodeID': 12, 'text': 'Default Illocuting', 'type': 'YA'}]

In [ ]:
dd = (['If the main Scottish elections return a full independence government Westminster government have xxx moral and legal right to deny another independence referendum It was by people uniting behind the Scottish Conservatives that stopped that SNP majority in 2016'])

In [ ]:
dddd = {"OVA": {}, 
        "text": " Stanley: <span class=\"highlighted\" id=\"0\">  Giles name for me please one thing that in your private life is utterly immoral but in politics would be regarded as a virtue? </span>.<br><br> Fraser: <span class=\"highlighted\" id=\"1\">  Well he said that ruthlessness was a classic example and thats what he focused on.   So ruthlessness is not something that is a domestic virtue know...</span>.<br><br>", 
        "AIF": {"schemefulfillments": [], 
                "nodes": [
                    {"text": "  Giles name for me please one thing that in your private life is utterly immoral but in politics would be regarded as a virtue? ", "type": "L", "nodeID": 1}, 
                    {"text": "  Giles name for me please one thing that in your private life is utterly immoral but in politics would be regarded as a virtue? ", "type": "I", "nodeID": 3}, 
                    {"text": "Default Illocuting", "type": "YA", "nodeID": 4}, 
                    {"text": "  Well he said that ruthlessness was a classic example and thats what he focused on.   So ruthlessness is not something that is a domestic virtue know...", "type": "L", "nodeID": 9}, 
                    {"text": "  Well he said that ruthlessness was a classic example and thats what he focused on.   So ruthlessness is not something that is a domestic virtue know...", "type": "I", "nodeID": 11}, 
                    {"text": "Default Illocuting", "type": "YA", "nodeID": 12}], 
                "participants": null, "locutions": [{"personID": "Stanley:", "nodeID": 1}, {"personID": "Fraser:", "nodeID": 9}], "descriptorfulfillments": [], "edges": [{"fromID": 1, "toID": 4, "edgeID": 5}, {"fromID": 4, "toID": 3, "edgeID": 6}, {"fromID": 9, "toID": 12, "edgeID": 13}, {"fromID": 12, "toID": 11, "edgeID": 14}]}}

In [73]:
from segmenter import Segmenter

ModuleNotFoundError: No module named 'util'

In [ ]:
{"AIF": {"descriptorfulfillments": [], 
         "edges": [{"edgeID": 5, "fromID": 1, "toID": 4}, {"edgeID": 6, "fromID": 4, "toID": 3},
                   {"edgeID": 12, "fromID": 8, "toID": 11}, {"edgeID": 13, "fromID": 11, "toID": 10},
                   {"edgeID": 19, "fromID": 15, "toID": 18}, {"edgeID": 20, "fromID": 18, "toID": 17}], 
         "locutions": [{"nodeID": 1, "personID": "Stanley:"}, {"nodeID": 8, "personID": "Fraser:"}, 
                       {"nodeID": 15, "personID": "Fraser:"}],
         "nodes": [{"nodeID": 4, "text": "Default Illocuting", "type": "YA"}, 
                   {"nodeID": 12, "text": "Default Illocuting", "type": "YA"},
                   {"nodeID": 1, "text": "Giles name for me please one thing that in your private life is utterly immoral but in politics would be regarded as a virtue ?", "type": "L"}, 
                   {"nodeID": 3, "text": "Giles name for me please one thing that in your private life is utterly immoral but in politics would be regarded as a virtue ?", "type": "I"}, 
                   {"nodeID": 4, "text": "Default Illocuting", "type": "YA"}, 
                   {"nodeID": 8, "text": "So ruthlessness is not something that is a domestic virtue know ...", "type": "L"}, 
                   {"nodeID": 10, "text": "So ruthlessness is not something that is a domestic virtue know ...", "type": "I"}, 
                   {"nodeID": 11, "text": "Default Illocuting", "type": "YA"}, 
                   {"nodeID": 15, "text": "Well he said that ruthlessness was a classic example and thats what he focused on", "type": "L"}, 
                   {"nodeID": 17, "text": "Well he said that ruthlessness was a classic example and thats what he focused on", "type": "I"}, 
                   {"nodeID": 18, "text": "Default Illocuting", "type": "YA"}], 
         "participants": null, "schemefulfillments": []},
 "OVA": {}, "text": " Stanley: <span class=\"highlighted\" id=\"0\">  Giles name for me please one thing that in your private life is utterly immoral but in politics would be regarded as a virtue? </span>.<br><br> Fraser: <span class=\"highlighted\" id=\"1\">  Well he said that ruthlessness was a classic example and thats what he focused on.   So ruthlessness is not something that is a domestic virtue know...</span>.<br><br>"}

In [ ]:
nodes =  [{"nodeID": "4_164761847968435155", "text": "Pure Questioning", "type": "YA"}, 
          {"nodeID": "5_164761847968435155", "text": "If the main Scottish elections return a full independence government, Westminster government have xxx moral and legal right to deny another independence referendum", "type": "I"}, 
          {"nodeID": "9_164761847968435155", "text": "Asserting", "type": "YA"},
          {"nodeID": "10_164761847968435155", "text": "Douglas Ross wants to stop a pro-independence majority government", "type": "I"},
          {"nodeID": "14_164761847968435155", "text": "Default Transition", "type": "TA"}, 
          {"nodeID": "16_164761847968435155", "text": "Asserting", "type": "YA"},
          {"nodeID": "17_164761847968435155", "text": "Douglas Ross wants to stop the SNP having a majority at this election, as the Scottish Conservatives did five years ago", "type": "I"},
          {"nodeID": "21_164761847968435155", "text": "Asserting", "type": "YA"},
          {"nodeID": "22_164761847968435155", "text": "It was by people uniting behind the Scottish Conservatives that stopped that SNP majority in 2016", "type": "I"}, 
          {"nodeID": "26_164761847968435155", "text": "Default Transition", "type": "TA"}, 
          {"nodeID": "28_164761847968435155", "text": "Default Rephrase", "type": "MA"},
          {"nodeID": "30_164761847968435155", "text": "Restating", "type": "YA"}, 
          {"nodeID": "32_164761847968435155", "text": "Default Transition", "type": "TA"},
          {"nodeID": "34_164761847968435155", "text": "Asserting", "type": "YA"}, 
          {"nodeID": "35_164761847968435155", "text": "people can unite again at this election to stop the SNP majority", "type": "I"},
          {"nodeID": "39_164761847968435155", "text": "Default Transition", "type": "TA"},
          {"nodeID": "41_164761847968435155", "text": "Default Inference", "type": "RA"}, 
          {"nodeID": "43_164761847968435155", "text": "Arguing", "type": "YA"},
          {"nodeID": 1, "text": "If the main Scottish elections return a full independence government what moral and legalright does the Westminster government have to deny another independence referendum", "type": "L"},
          {"nodeID": 3, "text": "If the main Scottish elections return a full independence government what moral and legalright does the Westminster government have to deny another independence referendum", "type": "I"},
          {"nodeID": 4, "text": "Default Illocuting", "type": "YA"},
          {"nodeID": 8, "text": "Blair Unknown : If the main Scottish elections return a full independence government what moral and legal right does the Westminster government have to deny another independence referendum", "type": "L"},
          {"nodeID": 10, "text": "Blair Unknown : If the main Scottish elections return a full independence government what moral and legal right does the Westminster government have to deny another independence referendum", "type": "I"},
          {"nodeID": 11, "text": "Default Illocuting", "type": "YA"},
          {"nodeID": 15, "text": "Kamila : Blair Unknown : If the main Scottish elections return a full independence government what moral and legal right does the Westminster government have to deny another independence referendum", "type": "L"}, 
          {"nodeID": 17, "text": "Kamila : Blair Unknown : If the main Scottish elections return a full independence government what moral and legal right does the Westminster government have to deny another independence referendum", "type": "I"},
          {"nodeID": 18, "text": "Default Illocuting", "type": "YA"},
          {"nodeID": 22, "text": "Douglas Ross : I want to stop a pro-independence majority government", "type": "L"}, 
          {"nodeID": 24, "text": "Douglas Ross : I want to stop a pro-independence majority government", "type": "I"}, 
          {"nodeID": 25, "text": "Default Illocuting", "type": "YA"}, 
          {"nodeID": 29, "text": "Kamila : Douglas Ross : I want to stop a pro-independence majority government", "type": "L"}, 
          {"nodeID": 31, "text": "Kamila : Douglas Ross : I want to stop a pro-independence majority government", "type": "I"}, 
          {"nodeID": 32, "text": "Default Illocuting", "type": "YA"},
          {"nodeID": 36, "text": "Douglas Ross : I want to stop the SNP having a majority at this election as the Scottish Conservatives did five years ago", "type": "L"},
          {"nodeID": 38, "text": "Douglas Ross : I want to stop the SNP having a majority at this election as the Scottish Conservatives did five years ago", "type": "I"}, 
          {"nodeID": 39, "text": "Default Illocuting", "type": "YA"},
          {"nodeID": 43, "text": "Kamila : Douglas Ross : I want to stop the SNP having a majority at this election as the Scottish Conservatives did five years ago", "type": "L"},
          {"nodeID": 45, "text": "Kamila : Douglas Ross : I want to stop the SNP having a majority at this election as the Scottish Conservatives did five years ago", "type": "I"}, 
          {"nodeID": 46, "text": "Default Illocuting", "type": "YA"}, {"nodeID": 50, "text": "Douglas Ross : It was by people uniting behind the Scottish Conservatives that stopped that SNP majority in 2016", "type": "L"}, {"nodeID": 52, "text": "Douglas Ross : It was by people uniting behind the Scottish Conservatives that stopped that SNP majority in 2016", "type": "I"}, {"nodeID": 53, "text": "Default Illocuting", "type": "YA"}, {"nodeID": 57, "text": "Kamila : Douglas Ross : It was by people uniting behind the Scottish Conservatives that stopped that SNP majority in 2016", "type": "L"}, {"nodeID": 59, "text": "Kamila : Douglas Ross : It was by people uniting behind the Scottish Conservatives that stopped that SNP majority in 2016", "type": "I"}, {"nodeID": 60, "text": "Default Illocuting", "type": "YA"}, {"nodeID": 64, "text": "Douglas Ross : people can do that again at this election", "type": "L"}, {"nodeID": 66, "text": "Douglas Ross : people can do that again at this election", "type": "I"}, {"nodeID": 67, "text": "Default Illocuting", "type": "YA"}, {"nodeID": 71, "text": "Kamila : Douglas Ross : people can do that again at this election", "type": "L"}, {"nodeID": 73, "text": "Kamila : Douglas Ross : people can do that again at this election", "type": "I"}, {"nodeID": 74, "text": "Default Illocuting", "type": "YA"}]

In [78]:
dct = {'a':1,'b':1, 'c':1}

print(list(dct.keys()))

['a', 'b', 'c']


In [80]:
if dct.get('d'):
    print("yes")

In [81]:
text = {
    "txt": "Fiona Bruce[00:01:10] Our first question tonight is from Blair Wondin.<div><br></div><div>Blair Unknown[00:01:22] Hello. <span class=\"highlighted\" id=\"node1\"><span class=\"highlighted\" id=\"node3_164761847968435155\">If the main Scottish elections return a full independence government, what moral and legal right does the Westminster government have to deny another independence referendum</span></span>?</div><div><br></div><div>Fiona Bruce[00:01:32] Doug?</div><div><br></div><div>Douglas Ross[00:01:33] Well, <span class=\"highlighted\" id=\"node8_164761847968435155\">I want to stop a pro-independence majority government</span>, <span class=\"highlighted\" id=\"node15_164761847968435155\">I want to stop the SNP having a majority at this election, as the Scottish Conservatives did five years ago</span>. <span class=\"highlighted\" id=\"node20_164761847968435155\">It was by people uniting behind the Scottish Conservatives that stopped that SNP majority in 2016</span>, and <span class=\"highlighted\" id=\"node33_164761847968435155\">people can do that again at this election</span>. </div>",
    "url": ""
}

print(text['txt'])
#if isinstance(text, dict):
    
    

Fiona Bruce[00:01:10] Our first question tonight is from Blair Wondin.<div><br></div><div>Blair Unknown[00:01:22] Hello. <span class="highlighted" id="node1"><span class="highlighted" id="node3_164761847968435155">If the main Scottish elections return a full independence government, what moral and legal right does the Westminster government have to deny another independence referendum</span></span>?</div><div><br></div><div>Fiona Bruce[00:01:32] Doug?</div><div><br></div><div>Douglas Ross[00:01:33] Well, <span class="highlighted" id="node8_164761847968435155">I want to stop a pro-independence majority government</span>, <span class="highlighted" id="node15_164761847968435155">I want to stop the SNP having a majority at this election, as the Scottish Conservatives did five years ago</span>. <span class="highlighted" id="node20_164761847968435155">It was by people uniting behind the Scottish Conservatives that stopped that SNP majority in 2016</span>, and <span class="highlighted" id="no

In [84]:
participants = dct.get("ff")
print(participants)
if not participants:
    participants = []

None


In [83]:
participants

[]